In [33]:
%load_ext autoreload
%autoreload 2

# Had issues pip installing psycopg2
# Run following command to resolve:
# env LDFLAGS="-I/usr/local/opt/openssl/include -L/usr/local/opt/openssl/lib" pip install psycopg2
# ref: https://stackoverflow.com/questions/39767810/cant-install-psycopg2-package-through-pip-install-is-this-because-of-sierra
import psycopg2
import csv
from psycopg2 import OperationalError

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
def create_connection(
    db_name, db_user, db_password, 
    db_host, db_port):
    """
    Establishes a connection with a PostgreSQL database
    
    Parameters:
    -------------
    db_name: str - Name of database
    db_user: str - Name of user
    db_password: str - Database password
    db_host: str - e.g. 127.0.0.1 (localhost)
    db_port: str - default 5432
    
    Returns:
    -------------
    n/a
    """
        
    connection = None
    
    try:
        connection = psycopg2.connect(
            database=db_name,
            user=db_user,
            password=db_password,
            host=db_host,
            port=db_port
        )
        print("Connection to PostgreSQL DB successful")
        
    except OperationalError as e:
        print(f"The error '{e}' occurred")
    
    return connection


def execute_query(connection, query):
    """
    Submit a query to a PostgreSQL database
    
    Parameters:
    -------------
    connection: psycopg2 object - Can create one using create_conneciton
    query: str - Any PostgreSQL query as a string
    
    Returns:
    -------------
    n/a
    """
    
    connection.autocommit = True
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Query executed succesfully")
    except OperationalError as e:
        print(f"The error '{e}' occurred")
        
        
def create_database(connection, db_name):
    """
    Creates a database if it does not already exist
    
    Parameters:
    -------------
    connection: psycopg2 object - Can create one using create_conneciton
    db_name: str - Name of database to be created
    
    Returns:
    -------------
    n/a
    """
    
    query_check = f"SELECT 1 FROM pg_catalog.pg_database WHERE datname = \'{db_name}\'"
    connection.autocommit = True
    cursor = connection.cursor()
    cursor.execute(query_check)
    exists = cursor.fetchone()
    
    if not exists:
        cursor.execute(f"CREATE DATABASE {db_name}")
    else:
        print(f"Database of {db_name} already exists!")

In [28]:
pw = input("Password:")

connection_init = create_connection(
    "postgres", "postgres", pw, "127.0.0.1", "5432"
)

# dbname = "medwatch"
# # Check if database exists
# query = f"""select exists(
#     SELECT datname FROM pg_catalog.pg_database WHERE lower(datname) = lower(\'{dbname}\')
# );
# """

# execute_query(connection_init, query)

Password:
Connection to PostgreSQL DB successful


In [29]:
create_database(connection_init, 'medwatch')

connection_init.close()

connection = create_connection(
    "medwatch", "postgres", pw, "127.0.0.1", "5432"
)

Database of medwatch already exists!
Connection to PostgreSQL DB successful


In [44]:
create_companies_table = """
CREATE TABLE IF NOT EXISTS companies (
  company TEXT,
  company_yahoo TEXT,
  symbol TEXT,
  exchange TEXT, 
  market_cap INT8,
  company_size TEXT,
  is_american BOOL,
  url_home TEXT,
  url_pr TEXT
)
"""

execute_query(connection, create_companies_table)

InterfaceError: connection already closed

In [43]:
# entry = [
#     ("AstraZeneca", "AstraZeneca PLC", "AZN", "NYSE", 139802e11, "large", True, "http://www.astrazeneca.com", "https://www.astrazeneca.com/media-centre/press-releases.html"),
#     ("Sinovac", "Sinovac Biotech Ltd.", "SVA", "NASDAQ", 460249280, "small", True, "http://www.sinovacbio.com", "http://www.sinovacbio.com/?optionid=754")
# ]

entry = []
with open('../datasets/updated_company_list.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    next(readCSV) # Skip header
    
    for row in readCSV:
        for i, element in enumerate(row):
            if element == 'n/a':
                row[i] = None
            elif i == 4:
                row[i] = int(float(element))
            elif i == 6:
                if element == 'Y':
                    row[i] = True
                elif element == 'N':
                    row[i] = False
                else:
                    row[i] = False
                    print('Not a valid Y or N argument')
            
        entry.append(tuple(row))

print(entry)

entry_records = ", ".join(["%s"]*len(entry))

insert_query = (
    f"INSERT INTO  companies (company, company_yahoo, symbol, exchange, market_cap, company_size, is_american, url_home, url_pr) VALUES {entry_records}"
)

connection.autocommit = True
cursor = connection.cursor()
cursor.execute(insert_query, entry)

[('Medicago Inc.', None, None, None, None, None, None, None, None), ('AstraZeneca', 'AstraZeneca PLC', 'AZN', 'NYSE', 139802000000, 'large', True, 'http://www.astrazeneca.com', 'https://www.astrazeneca.com/media-centre/press-releases.html'), ('Cadila Healthcare Limited', None, None, None, None, None, None, None, None), ('Beijing Institute of Biotechnology', None, None, None, None, None, None, None, None), (' Takara Bio', 'Takara Bio Inc.', 'TKBIF', 'OTC Markets', None, None, False, 'http://www.takara-bio.com', 'http://www.takara-bio.com/release/'), ('Wuhan Institute of Biological Products', None, None, None, None, None, None, None, None), ('Beijing Institute of Biological Products', None, None, None, None, None, None, None, None), ('Sinovac', 'Sinovac Biotech Ltd.', 'SVA', 'NASDAQ', 460249280, 'small', True, 'http://www.sinovacbio.com', 'http://www.sinovacbio.com/?optionid=754'), ('Gamaleya Research Institute', None, None, None, None, None, None, None, None), ('Moderna', 'Moderna, Inc.

In [32]:
connection.close()